Karol Janic

kwiecień 2023

zad3.ipynb - Minimalizacja liczby radiowozów w pewnym mieście.

In [1]:
using JuMP
using GLPK

using JSON

In [2]:
dane_wejsciowe = JSON.parse(read("data3.json", String))

dzielnice = dane_wejsciowe["dzielnice"]
zmiany = dane_wejsciowe["zmiany"]
ograniczenia = dane_wejsciowe["ograniczenia"]

Dict{String, Any} with 3 entries:
  "dzielnica 1" => Dict{String, Any}("zmiana 2"=>Dict{String, Any}("max"=>7, "m…
  "dzielnica 2" => Dict{String, Any}("zmiana 2"=>Dict{String, Any}("max"=>7, "m…
  "dzielnica 3" => Dict{String, Any}("zmiana 2"=>Dict{String, Any}("max"=>12, "…

In [3]:
model = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

Definicja zmiennych decyzyjnych(liczba radiowozów na danej zmianie w danej dzielnicy) oraz ograniczenie ich wartości do liczb nieujemnych:

In [4]:
@variable(model, x[keys(zmiany), keys(dzielnice)] >= 0)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, ["zmiana 2", "zmiana 1", "zmiana 3"]
    Dimension 2, ["dzielnica 1", "dzielnica 2", "dzielnica 3"]
And data, a 3×3 Matrix{VariableRef}:
 x[zmiana 2,dzielnica 1]  x[zmiana 2,dzielnica 2]  x[zmiana 2,dzielnica 3]
 x[zmiana 1,dzielnica 1]  x[zmiana 1,dzielnica 2]  x[zmiana 1,dzielnica 3]
 x[zmiana 3,dzielnica 1]  x[zmiana 3,dzielnica 2]  x[zmiana 3,dzielnica 3]

Definicja zmiennej decyzyjnej oznaczająca niezbędną liczbę radiowozów oraz ograniczenie jej wartości do liczb nieujemnych:

In [5]:
@variable(model, liczba_radiowozow >= 0)

liczba_radiowozow

Ograniczenie minimalnej liczby radiowozów w danej dzielnicy na danej zmianie:

In [6]:
@constraint(model, 
    [dzielnica in keys(dzielnice), zmiana in keys(zmiany)], 
    x[zmiana, dzielnica] >= ograniczenia[dzielnica][zmiana]["min"])

2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, ["dzielnica 1", "dzielnica 2", "dzielnica 3"]
    Dimension 2, ["zmiana 2", "zmiana 1", "zmiana 3"]
And data, a 3×3 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 x[zmiana 2,dzielnica 1] ≥ 4.0  …  x[zmiana 3,dzielnica 1] ≥ 3.0
 x[zmiana 2,dzielnica 2] ≥ 6.0     x[zmiana 3,dzielnica 2] ≥ 5.0
 x[zmiana 2,dzielnica 3] ≥ 7.0     x[zmiana 3,dzielnica 3] ≥ 6.0

Ograniczenie maksymalnej liczby radiowozów w danej dzielnicy na danej zmianie:

In [7]:
@constraint(model, 
    [dzielnica in keys(dzielnice), zmiana in keys(zmiany)], 
    x[zmiana, dzielnica] <= ograniczenia[dzielnica][zmiana]["max"])

2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, ["dzielnica 1", "dzielnica 2", "dzielnica 3"]
    Dimension 2, ["zmiana 2", "zmiana 1", "zmiana 3"]
And data, a 3×3 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 x[zmiana 2,dzielnica 1] ≤ 7.0   …  x[zmiana 3,dzielnica 1] ≤ 5.0
 x[zmiana 2,dzielnica 2] ≤ 7.0      x[zmiana 3,dzielnica 2] ≤ 10.0
 x[zmiana 2,dzielnica 3] ≤ 12.0     x[zmiana 3,dzielnica 3] ≤ 10.0

Ograniczenie minimalnej liczby radiowozów w danej dzielnicy

In [8]:
@constraint(model, 
    [dzielnica in keys(dzielnice)], sum(x[:, dzielnica]) >= dzielnice[dzielnica]["obsada"])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["dzielnica 1", "dzielnica 2", "dzielnica 3"]
And data, a 3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 x[zmiana 2,dzielnica 1] + x[zmiana 1,dzielnica 1] + x[zmiana 3,dzielnica 1] ≥ 10.0
 x[zmiana 2,dzielnica 2] + x[zmiana 1,dzielnica 2] + x[zmiana 3,dzielnica 2] ≥ 14.0
 x[zmiana 2,dzielnica 3] + x[zmiana 1,dzielnica 3] + x[zmiana 3,dzielnica 3] ≥ 13.0

Ograniczenie minimalnej liczby radiowozów na danej zmianie:

In [9]:
@constraint(model, 
    [zmiana in keys(zmiany)], sum(x[zmiana, :]) >= zmiany[zmiana]["obsada"])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["zmiana 2", "zmiana 1", "zmiana 3"]
And data, a 3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 x[zmiana 2,dzielnica 1] + x[zmiana 2,dzielnica 2] + x[zmiana 2,dzielnica 3] ≥ 20.0
 x[zmiana 1,dzielnica 1] + x[zmiana 1,dzielnica 2] + x[zmiana 1,dzielnica 3] ≥ 10.0
 x[zmiana 3,dzielnica 1] + x[zmiana 3,dzielnica 2] + x[zmiana 3,dzielnica 3] ≥ 18.0

Ograniczenie niezbędnej liczby radiowozów:

In [10]:
@constraint(model, [zmiana in keys(zmiany)], liczba_radiowozow >= sum(x[zmiana, :]))

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["zmiana 2", "zmiana 1", "zmiana 3"]
And data, a 3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 -x[zmiana 2,dzielnica 1] - x[zmiana 2,dzielnica 2] - x[zmiana 2,dzielnica 3] + liczba_radiowozow ≥ 0.0
 -x[zmiana 1,dzielnica 1] - x[zmiana 1,dzielnica 2] - x[zmiana 1,dzielnica 3] + liczba_radiowozow ≥ 0.0
 -x[zmiana 3,dzielnica 1] - x[zmiana 3,dzielnica 2] - x[zmiana 3,dzielnica 3] + liczba_radiowozow ≥ 0.0

Funkcja kosztu(niezbędna liczba radiowozów):

In [11]:
@objective(model, Min, liczba_radiowozow)

liczba_radiowozow

In [12]:
optimize!(model)

In [13]:
solution_summary(model)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 2.00000e+01
  Objective bound    : -Inf
  Dual objective value : 2.00000e+01

* Work counters
  Solve time (sec)   : 2.31028e-04


In [14]:
JuMP.objective_value(model)

20.0

In [15]:
println("Liczba radiowozow: ", value(liczba_radiowozow))

for dzielnica in sort!(collect(keys(dzielnice))), zmiana in sort!(collect(keys(zmiany)))
    println(dzielnica, " i ", zmiana, ": ", value(x[zmiana, dzielnica]))
end

Liczba radiowozow: 20.0
dzielnica 1 i zmiana 1: 3.0
dzielnica 1 i zmiana 2: 4.0
dzielnica 1 i zmiana 3: 5.0
dzielnica 2 i zmiana 1: 3.0
dzielnica 2 i zmiana 2: 7.0
dzielnica 2 i zmiana 3: 7.0
dzielnica 3 i zmiana 1: 8.0
dzielnica 3 i zmiana 2: 9.0
dzielnica 3 i zmiana 3: 6.0
